# Pointer Generator Model

In [1]:
import torch
from PICOHelper import get_pico_datasets
from NewsroomHelper import get_newsroom_datasets
from models import Summarizer
from model_helpers import loss_function, error_function
from utils import get_index_words, produce_attention_visualization_file, summarize, get_text_triplets, produce_batch_summary_files, produce_summary_files
from pytorch_helper import VariableBatchDataLoader, ModelManipulator, plot_learning_curves

## Parameters

In [2]:
# training parameters
BATCH_SIZE = 64
NUM_EPOCHS = 2
LEARNING_RATE = 1e-2
# INITIAL_ACCUMULATOR_VALUE = 0.1
GAMMA = 1
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)
BEAM_SIZE = 4

True


## Get Data

In [3]:
# pico_dataset_train, pico_dataset_dev, pico_dataset_test = get_pico_datasets()
newsroom_dataset_train, newsroom_dataset_dev, newsroom_dataset_test, preprocessor = get_newsroom_datasets(with_oov=True)
word_vectors = preprocessor.word_vectors
start_index = preprocessor.word_indices['<start>']
end_index = preprocessor.word_indices['<end>']

11029 3676 3678
retrieving word2vec model from file


## Create Model

In [ ]:
pointer_generator_model = Summarizer(preprocessor, start_index, end_index, num_hidden1=None, num_hidden2=None, with_coverage=True, gamma=GAMMA, with_pointer=True)

## Train and Save Model

In [ ]:
dataloader = VariableBatchDataLoader(newsroom_dataset_train, batch_size=BATCH_SIZE, shuffle=True)
optimizer = torch.optim.Adam(pointer_generator_model.parameters(),
                             lr=LEARNING_RATE)
# optimizer = torch.optim.Adagrad((pointer_generator_model.cuda() if USE_CUDA else pointer_generator_model).parameters(),
#                                 lr=LEARNING_RATE, initial_accumulator_value=INITIAL_ACCUMULATOR_VALUE)
model_manip = ModelManipulator(pointer_generator_model, optimizer, loss_function, error_function, use_cuda=USE_CUDA)
train_stats, val_stats = model_manip.train(dataloader, NUM_EPOCHS, dataset_val=newsroom_dataset_dev, stats_every=10, verbose_every=10)

In [ ]:
torch.save(pointer_generator_model, 'models/pointer_generator_temp.model')

In [4]:
pointer_generator_model = torch.load('models/pointer_generator_temp.model')

## Plot

In [ ]:
plot_learning_curves(training_values=train_stats, validation_values=val_stats, figure_name='graphs/pointer_generator_training_temp')

In [ ]:
batch = newsroom_dataset_dev[20:25]
results = summarize(batch, pointer_generator_model, beam_size=BEAM_SIZE)

In [ ]:
summary_info = results[0]
triplets = get_text_triplets(batch, summary_info, preprocessor, pointer_gen=True)
for i,(text, reference_summary, decoded_summary) in enumerate(triplets):
    loss = summary_info[2][i]
    print("text", text)
    print("reference summary", reference_summary)
    print("decoded summary", decoded_summary)
    print(loss)

In [ ]:
summary_info = results[0]
i = 2

triplets = get_text_triplets(batch, summary_info, preprocessor)

text, reference_summary, decoded_summary = triplets[i]
attentions, p_gens = [[float(f) for f in vector[1:-1]] for vector in summary_info[4][i][:-1]], [float(f) for f in summary_info[5][i][:-1]]

produce_attention_visualization_file('graphs/attn_vis_data.json', text, " ".join(reference_summary), decoded_summary, attentions, p_gens)

In [ ]:
produce_batch_summary_files(batch, pointer_generator_model, "rouge", beam_size=BEAM_SIZE)

In [17]:
dataloader = VariableBatchDataLoader(newsroom_dataset_train, batch_size=5, shuffle=True)
produce_summary_files(dataloader, pointer_generator_model, "rouge", beam_size=BEAM_SIZE, max_num_batch=10)

0
1
2
3
4
5
6
7
8
9


In [18]:
import pandas as pd
df = pd.read_csv("rouge/ROUGE-2/results.csv")
df

,ROUGE-Type,Task Name,System Name,Avg_Recall,Avg_Precision,Avg_F-Score,Num Reference Summaries
0,ROUGE-L+StopWordRemoval,ARTICLE36,SYSTEM.TXT,0.12500,0.08333,0.10000,1
1,ROUGE-1+StopWordRemoval,ARTICLE36,SYSTEM.TXT,0.12500,0.08333,0.10000,1
2,ROUGE-2+StopWordRemoval,ARTICLE36,SYSTEM.TXT,0.00000,0.00000,0.00000,1
3,ROUGE-SU4+StopWordRemoval,ARTICLE36,SYSTEM.TXT,0.03333,0.02000,0.02500,1
4,ROUGE-L+StopWordRemoval,ARTICLE35,SYSTEM.TXT,0.06250,0.10000,0.07692,1
5,ROUGE-1+StopWordRemoval,ARTICLE35,SYSTEM.TXT,0.06250,0.10000,0.07692,1
6,ROUGE-2+StopWordRemoval,ARTICLE35,SYSTEM.TXT,0.00000,0.00000,0.00000,1
7,ROUGE-SU4+StopWordRemoval,ARTICLE35,SYSTEM.TXT,0.01429,0.02500,0.01818,1
8,ROUGE-L+StopWordRemoval,ARTICLE38,SYSTEM.TXT,0.64286,0.52941,0.58065,1
9,ROUGE-1+StopWordRemoval,ARTICLE38,SYSTEM.TXT,0.64286,0.52941,0.58065,1


In [19]:
df[df['ROUGE-Type'] == 'ROUGE-L+StopWordRemoval']['Avg_F-Score'].mean()

0.13188730000000004